In [1]:
# Importa as bibliotecas necessárias
%run '~/git/Bibliotecas.ipynb'

# Importação

In [2]:
# Carrega a base de dados contendo os registros de importação de fertilizantes
im=pd.read_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/i.csv').drop(columns='date')

# Elimina ano incompleto
im=im[im.CO_ANO!=2024]

# Modifica unidade de peso
im.KG_LIQUIDO=im.KG_LIQUIDO/10**6
im=im.rename(columns={'KG_LIQUIDO':'kton'})
im.head(1)

,CO_ANO,CO_MES,kton
0,1997,3,7.505776


# Chuvas

In [3]:
ch=pd.read_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/c.csv').drop(columns='date')
ch=ch[ch.CO_ANO!=2001]
ch.tail(1)

,CO_ANO,CO_MES,mm
264,2023,12,78.6


# Teste de hipóteses:

Em relação ao o erro médio absoluto do modelo preditivo, o uso dos registros de chuva como input adicional:

* Ho = "Não altera o erro médio absoluto do modelo preditivo"

* Ha = "Altera o erro médio absoluto do modelo preditivo"


Serão testados paralelamente:
* Níveis de hyperparâmetros do algoritmo regressor (md)
* Espaço de tempo (meses) entre uma determinada chuva até sua possível influência nos desembarques (ad)
* Meses a serem utilizadas como treino (train_size_mes)

Executa os testes

In [ ]:
q=pd.DataFrame()

for md in [None,1]:
    
    from sklearn.ensemble import RandomForestRegressor as RFR
    RFR=RFR(max_depth=md)
    
    for ad in [7,8]:
    

        i=im.copy()
        c=ch.copy()
    
        c.CO_MES=c.CO_MES+ad
        for adi in range(1,ad+1):
            c.loc[c.CO_MES==12+adi,'CO_ANO']=c.loc[c.CO_MES==12+adi]['CO_ANO']+1
            c.loc[c.CO_MES==12+adi,'CO_MES']=adi
        
        i=i.merge(c)

        i['date']=pd.to_datetime(i.CO_ANO.astype(str)+'/'+i.CO_MES.astype(str)+'/1').astype(int)
        i=i.drop(columns='CO_ANO')
        i.CO_MES=i.CO_MES.astype('category')
        i=pd.get_dummies(i)
        
        for a in range(1,len(i)):
    
        
            for train_size_mes in [5*12,8*12]:

    
                train=i.iloc[:a,:].tail(train_size_mes)
                
                if len(train)>=train_size_mes:

    
                    
                    test=i.iloc[a:a+1,:]
                                
                    # Train
                    X_train_mm=train.drop(columns='kton')
                    X_train_s_mm=train.drop(columns=['kton','mm'])
                    y_train=train.kton
            
                    # Test
                    X_test_mm  =test.drop(columns='kton')
                    X_test_s_mm=test.drop(columns=['kton','mm'])
                    y_test=test.kton
                        
                    RFR.fit(X_train_mm,y_train)
                    
                    test_pred=test     .assign(kton_pred_mm  =RFR.predict(X_test_mm  ).astype(int))
                    
                    RFR.fit(X_train_s_mm,y_train)
                    
                    test_pred=test_pred.assign(kton_pred_s_mm=RFR.predict(X_test_s_mm).astype(int))
    
                    test_pred=test_pred.assign(ad=ad,train_size_anos=train_size_mes/12,md=md)
                    
                    q=pd.concat([q,test_pred])

q['erro_mm']=np.abs(q.kton-q.kton_pred_mm)
q['erro_s_mm']=np.abs(q.kton-q.kton_pred_s_mm)

print('erro_mm.mean',round(q.erro_mm.mean()))
print('erro_s_mm.mean',round(q.erro_s_mm.mean()))

Salva o resultado dos testes

In [7]:
q.to_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/q.csv', index=False)